# Step1: こちらのセルを実行し、認証を通してください

In [1]:
!pip install --upgrade --quiet google-cloud-aiplatform google-genai
!pip install --upgrade -q google-api-python-client google-auth-httplib2 google-auth-oauthlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 52.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.8/222.8 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.7/13.7 MB 105.1 MB/s eta 0:00:00


In [2]:
import os
import re
import glob
import numpy as np
import pandas as pd
from google import genai
from google.genai import types
import json
import seaborn as sns
!pip install japanize-matplotlib
import matplotlib.pyplot as plt
import japanize_matplotlib
!pip install openpyxl
from tqdm import tqdm
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
import io

from google.colab import drive
drive.mount('/content/drive')
from natsort import natsorted

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 18.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for japanize-matplotlib: filename=japanize_matplotlib-1.1.3-py3-none-any.whl size=4120257 sha256=a6fe4f869af72e73750dd40965b5d7650acde567725c063aea6e9e83d9644078
  Stored in directory: /root/.cache/pip/wheels/da/a1/71/b8faeb93276fed10edffcca20746f1ef6f8d9e071eee8425fc
Successfully built japanize-matplotlib
Mounted at /content/drive


1. 下記リンクの共有フォルダのショートカットを「マイドライブ」に作成
https://drive.google.com/drive/folders/1PIFRHDW4Ra_gWnqkxj7uA5gcQLUxZWWq
<br>※「マイドライブ」に作成しない場合、次セルでエラーが発生する

In [3]:
import sys

if "google.colab" in sys.modules:

    from google.colab import auth

    auth.authenticate_user()

In [ ]:
import gspread
from gspread_dataframe import set_with_dataframe
# google.authからdefaultをインポートします
from google.auth import default

# Google Colab環境での認証を実行します
# 実行後、表示されるURLにアクセスし、認証コードをコピーして入力してください
auth.authenticate_user()

# 新しい推奨される方法で認証情報を取得し、gspreadを初期化します
creds, _ = default()
gc = gspread.authorize(creds)

print("Googleアカウントでの認証が完了しました。")

# Step2: 中間・最終アンケートのspreadsheetの場合はsheet_id、sheet名を入力してください。また、整形後の表データを格納したいsheet名も記入してください。

In [4]:
sheet_id = '' # @param {type:"string"}
sheet_name = 'フォームの回答 1' # @param {type:"string"}
sheet_name_revised = '整形後' # @param {type:"string"}

In [6]:
# 認証済みのDrive APIクライアント
drive_service = build('drive', 'v3')

# スプレッドシートを.xlsx形式でエクスポート
request = drive_service.files().export_media(
    fileId=sheet_id,
    mimeType='application/vnd.openxmlformats-officedocument.spreadsheetml.sheet'
)

# メモリ上でファイルとして保持
fh = io.BytesIO()
downloader = MediaIoBaseDownload(fh, request)

done = False
while not done:
    status, done = downloader.next_chunk()

# 読み込み位置を巻き戻して、pandasで読み込む
fh.seek(0)
df = pd.read_excel(fh, sheet_name=sheet_name)

# Step3 : 以下自動で実行します。

In [ ]:
#object型の列の内容の一致率で閾値を超えたものは選択式の設問だと判断
SIMILARITY_THRESHOLD = 0.1 # @param {type:"number"}
#MINORITY_THRESHOLD回しか出現しなかった回答が「その他」に集約されます。(「5」くらいがちょうどよいです！)
MINORITY_THRESHOLD = 5 # @param {type:"integer"}

In [7]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)

In [8]:
df.head()

,タイムスタンプ,OmnicampusアカウントID,Omnicampusアカウント名,GCI受講の意欲について教えてください。,上記回答の理由・動機などについて簡単に教えてください。,GCI受講継続の妨げとなっている点・難しいと感じる点はありますか？（複数選択可）,上記回答を解決するためにどんなサポート・改善があるといいですか？ ご自由にご記入ください。,GCI受講にLLM(ChatGPT等)を活用していますか？,LLM(ChatGPT等)をどんな用途で使用していますか？（複数選択可）,Slackの活用について難しく感じる部分はありますか？（複数選択可）,疑問解決にSlackのカスタムレスポンスを活用していますか？,受講の手引き・Slackでの情報アクセスはスムーズ/わかりやすいですか？,情報のキャッチアップにおいてどんな点で困ったことがありますか？ご自由にご記入下さい。,より学びやすい受講環境のためにどんなサポート・改善があるといいですか？ ご自由にご記入ください。,Botからの通知はいかがでしょうか？(複数選択可),どんなメッセージや機能があると良いですか？ご自由にご記入ください。,講義冒頭の松尾先生が登場するGCI紹介動画の印象を教えて下さい。（複数選択可）,GCI紹介動画の次に流れる、TAからのメッセージの印象を教えて下さい。(複数選択可),講義資料に関して、わかりづらかった回があれば教えて下さい。（複数選択可）,上記で選択した回の講義資料について、わかりづらかった部分を教えて下さい。,上記の座談会に参加されましたか？,【参加した方に質問です】\n 感じたこと、良かったことがあれば教えて下さい。,その他、このような人から・このような話が聞きたいなどあれば教えて下さい。,オフィスアワーの参加経験(回数)について教えてください。,【オフィスアワーに参加したことがある方に質問です】\n参加した理由を教えてください。（複数選択可）,【オフィスアワーに参加したことがある方に質問です】\n参加して良かったことがあれば教えてください。,【オフィスアワーに参加したことがない方に質問です】\n参加されない理由を教えてください。（複数選択可）,オフィスアワーの開催テーマとして興味のあるものはなんですか？,オフィスアワーの質疑応答集シートを活用していますか？,【週末に開催している初心者向けOHに参加したことがある方に質問です】\n初心者向けOHの満足度はいかがでしょうか？（参加された方のみ）,オフィスアワーの改善点・要望などがあればご自由にご記入ください。,コンペ1に取り組んだおおよその時間数を教えて下さい。(半角数字),コンペdemoノートブック(biginner.ipynb / professional.ipynb)のわかりやすさはいかがでしたか？（利用された方のみ）,コンペに役立ったコンテンツをお選びください。（複数選択可）,上記の役立ったコンテンツについて、回答された理由をご記入ください。,コンペについての改善点・要望などがあればご自由にご記入ください。\n(demoノートブックにてわかりづらい点があればこちらに),最終課題の取り組み状況について教えてください。,最終課題の難しい/難しそうなポイントはどこですか？（複数選択可）,もくもく会の参加状況を教えてください。,もくもく会に望むこと・期待することについて教えて下さい。（複数選択可）,上記のTimesを参考にされていますか？,GCI受講体験に関して、改善点・要望などがあればご自由にご記入ください。,ご感想などあればご自由にご記入ください。
0,2025-06-17 20:07:35.875,64412,Snowman03,優秀修了を目指している,講座の内容が興味深かったため,"最終課題が難しい, 講義内容が難しい",オフィスアワーは良いサポートだと思う,活用している（GCI受講前から利用していた）,"講義内容の深堀り, コーディング補助, 講義内容の疑問解決","1万人規模のSlackでの発言がはばかられる, 情報量が多く、何を見たらいいかわからない",活用していない（存在を知らない）,4,NaN,NaN,Botからのメッセージが届いたことがない,NaN,あまり必要だと感じない,その他の理由から悪印象,"第7回 特徴量エンジニアリング（説明変数の変換・生成による精度向上）, 第8回 教師なし学習/時系列",NaN,両方参加した,NaN,NaN,2-4回参加した,"開催テーマに関心があった, 雰囲気を味わいたい / 作業時間確保のため, 学習方法やコツなどの講義内容以外に関する質問があった",NaN,NaN,"コンペ, 最終課題, 講義の振り返り",活用していない,NaN,NaN,1.0,5.0,"demoノートブック（biginner.ipynb / professional.ipynb）, オフィスアワー",NaN,NaN,全く手を付けていない,まだわからない,参加していない（今後も参加するかはわからない）,特にない,見たことがない,NaN,NaN
1,2025-06-17 20:08:45.737,43639,Masa2346,修了を目指している,AIに触れることが目標であったから。,特にない,特になし,活用している（GCI受講をきっかけに利用開始した）,コーディング補助,特に問題はない,活用していない（存在を知らない）,4,NaN,NaN,Botからのメッセージが届いたことがない,NaN,モチベーション維持につながっている,モチベーション維持につながっている,わかりづらい講義資料はなかった,NaN,参加しなかった,NaN,NaN,1回参加した,学習方法やコツなどの講義内容以外に関する質問があった,他の受講生の質問とその回答を聞き勉強になったこと,NaN,"コンペ, 最終課題",活用していない,NaN,NaN,1.0,5.0,"demoノートブック（biginner.ipynb / professional.ipynb）, 受講者同士のSlackでの疑問解決・議論",知りたい情報にすぐにアクセスできたから。,NaN,内容は確認したが取り組んでいない,"データ分析 / モデル構築, 事業提案, スライド作成",参加していない（今後参加してみたい）,TAによるサポート,見たことはあるが、参考にはしていない,NaN,NaN
2,2025-06-17 20:12:42.980,64042,Rikuto_s_,修了を目指している,業務と両立しながらの受講であるが、講義内容は興味があるため。,特にない,特にありません。,活用している（GCI受講前から利用していた）,"コーディング補助, 講義内容の疑問解決, 講義内容の深堀り, コンペ・最終課題の解法の壁打ち",特に問題はない,活用していない（存在を知っている）,4,NaN,NaN,Botからのメッセージが届いたことがない,NaN,動画の間に受講環境を準備している,講義の全体像がよくわかる,わかりづらい講義資料はなかった,NaN,参加しなかった,NaN,NaN,参加したことがない,NaN,NaN,困りごとや不安がないから,講義の振り返り,活用していない,NaN,NaN,20.0,5.0,"demoノートブック（biginner.ipynb / professional.ipynb）, README解説動画",NaN,１日の採点の回数が少ない,全く手を付けていない,まだわからない,参加していない（今後も参加するかはわからない）,特にない,見たことはあるが、参考にはしていない,NaN,NaN
3,2025-06-17 20:14:36.356,65276,zen-izuru,優秀修了を目指している,特典が魅力的であり、また今後のキャリアに活かせそうだから,特にない,特に困っていることはありません。,活用している（GCI受講前から利用していた）,"コーディング補助, コンペ・最終課題の解法の壁打ち, 講義内容の疑問解決, 講義内容の深堀り",特に問題はない,活用していない（存在を知っている）,5,NaN,NaN,現在の通知内容で満足している,NaN,"今から授業が始まるという心の準備につながっている, 東大松尾研の講義であることの認識につながっている","受講中のイメージがわく, 講義の全体像がよくわかる, 受講のヒントになっている, TAを身近に感じられる",わかりづらい講義資料はなかった,NaN,参加しなかった,NaN,NaN,参加したことがない,NaN,NaN,時間が合わなかったから,"コンペ, 最終課題",活用していない,NaN,NaN,50.0,5.0,"コンペマイルストーン, README解説動画, demoノートブック（biginner.ipynb / professional.ipynb）, 受講者同士のSlackでの疑問解決・議論",特にdemoはとても参考になりました。,NaN,内容は確認したが取り組んでいない,まだわからない,参加していない（今後も参加するかはわからない）,"他の受講生との交流, TAによるサポート",見たことがあり、参考にしている,特にありません。,全体的に満足しています。講義も教材も非常に役立っています。
4,2025-06-17 20:17:19.735,58054,Oimosan0141ne,修了を目指している,優秀修了を目指してたいが、本業が忙しくてそこまで時間がさけず非常に残念ですが、修了できるように専念しています

In [41]:
multi_choice_columns = []
column_scores = {}

for col in df.select_dtypes(include=['object']).columns:
    series = df[col].dropna()
    if not series.str.contains(',').any():
        continue
    all_elements = series.str.split(',').explode().str.strip().tolist()
    all_elements = [elem for elem in all_elements if elem]
    if not all_elements:
        continue
    total_elements = len(all_elements)
    unique_elements = len(set(all_elements))
    score = unique_elements / total_elements if total_elements > 0 else 1.0
    column_scores[col] = score
    if score <= SIMILARITY_THRESHOLD:
        multi_choice_columns.append(col)
if not multi_choice_columns:
    print("\n複数選択項目と判定される列はありませんでした。")
    raise ValueError("One-Hotエンコーディング対象の列が見つかりません。")
print("\n【最終結果】")
print("▼ 以下の列を複数選択項目として処理します：")
for col_name in multi_choice_columns:
    print(f"  - {col_name}")
print("-------------------------------------------------")

# 元のデータフレームから、One-Hot化する列を除いた基本データを作成
df_base = df.drop(columns=multi_choice_columns)

# One-Hot化したデータフレームを格納するリスト
one_hot_dfs = []

# 自動判定された各列に対してループ処理
for column_name in multi_choice_columns:
    # カンマで分割 -> explodeで縦持ちに -> 前後の空白削除
    all_choices = df[column_name].str.split(',').explode().str.strip()
    # 有効な文字列のみを対象にする
    all_choices = all_choices.dropna()[all_choices.dropna().str.len() > 0]

    value_counts = all_choices.value_counts()

    # 出現頻度が閾値以下の「少数意見」を特定
    minority_choices = set(value_counts[value_counts <= MINORITY_THRESHOLD].index)

    # 回答を書き換えるための関数を定義
    def clean_and_aggregate(answer_str):
        if pd.isna(answer_str):
            return ""

        choices = [s.strip() for s in str(answer_str).split(',')]

        # 少数意見、または元々「その他」で始まる記述を「その他」に統一
        processed_choices = [
            'その他' if choice.startswith('その他') or choice in minority_choices else choice
            for choice in choices if choice # 空の選択肢は無視
        ]

        # 重複を削除し、再度カンマで結合
        return ','.join(sorted(list(set(processed_choices))))

    #上記関数を適用してクレンジング
    cleaned_series = df[column_name].apply(clean_and_aggregate)

    #One-Hotエンコーディングを実行
    one_hot_df = cleaned_series.str.get_dummies(sep=',')

    # '' という名前の列が生成された場合は削除
    if '' in one_hot_df.columns:
        one_hot_df = one_hot_df.drop(columns=[''])

    one_hot_df.columns = one_hot_df.columns.str.strip()
    one_hot_df = one_hot_df.add_prefix(f"{column_name}_") # 列名の重複を防ぐ

    one_hot_dfs.append(one_hot_df)

# 全てのデータフレームを結合
df_new = pd.concat([df_base] + one_hot_dfs, axis=1)
df_new.columns = df_new.columns.str.replace(r'[\r\n]+', ' ', regex=True).str.strip()


【最終結果】
▼ 以下の列を複数選択項目として処理します：
  - GCI受講継続の妨げとなっている点・難しいと感じる点はありますか？（複数選択可）
  - LLM(ChatGPT等)をどんな用途で使用していますか？（複数選択可）
  - Slackの活用について難しく感じる部分はありますか？（複数選択可）
  - Botからの通知はいかがでしょうか？(複数選択可)
  - 講義冒頭の松尾先生が登場するGCI紹介動画の印象を教えて下さい。（複数選択可）
  - GCI紹介動画の次に流れる、TAからのメッセージの印象を教えて下さい。(複数選択可)
  - 講義資料に関して、わかりづらかった回があれば教えて下さい。（複数選択可）
  - 【オフィスアワーに参加したことがある方に質問です】
参加した理由を教えてください。（複数選択可）
  - 【オフィスアワーに参加したことがある方に質問です】
参加して良かったことがあれば教えてください。
  - 【オフィスアワーに参加したことがない方に質問です】
参加されない理由を教えてください。（複数選択可）
  - オフィスアワーの開催テーマとして興味のあるものはなんですか？
  - コンペに役立ったコンテンツをお選びください。（複数選択可）
  - 最終課題の難しい/難しそうなポイントはどこですか？（複数選択可）
  - もくもく会に望むこと・期待することについて教えて下さい。（複数選択可）
-------------------------------------------------


In [42]:
df_new.head()

,タイムスタンプ,OmnicampusアカウントID,Omnicampusアカウント名,GCI受講の意欲について教えてください。,上記回答の理由・動機などについて簡単に教えてください。,上記回答を解決するためにどんなサポート・改善があるといいですか？ ご自由にご記入ください。,GCI受講にLLM(ChatGPT等)を活用していますか？,疑問解決にSlackのカスタムレスポンスを活用していますか？,受講の手引き・Slackでの情報アクセスはスムーズ/わかりやすいですか？,情報のキャッチアップにおいてどんな点で困ったことがありますか？ご自由にご記入下さい。,より学びやすい受講環境のためにどんなサポート・改善があるといいですか？ ご自由にご記入ください。,どんなメッセージや機能があると良いですか？ご自由にご記入ください。,上記で選択した回の講義資料について、わかりづらかった部分を教えて下さい。,上記の座談会に参加されましたか？,【参加した方に質問です】 感じたこと、良かったことがあれば教えて下さい。,その他、このような人から・このような話が聞きたいなどあれば教えて下さい。,オフィスアワーの参加経験(回数)について教えてください。,オフィスアワーの質疑応答集シートを活用していますか？,【週末に開催している初心者向けOHに参加したことがある方に質問です】 初心者向けOHの満足度はいかがでしょうか？（参加された方のみ）,オフィスアワーの改善点・要望などがあればご自由にご記入ください。,コンペ1に取り組んだおおよその時間数を教えて下さい。(半角数字),コンペdemoノートブック(biginner.ipynb / professional.ipynb)のわかりやすさはいかがでしたか？（利用された方のみ）,上記の役立ったコンテンツについて、回答された理由をご記入ください。,コンペについての改善点・要望などがあればご自由にご記入ください。 (demoノートブックにてわかりづらい点があればこちらに),最終課題の取り組み状況について教えてください。,もくもく会の参加状況を教えてください。,上記のTimesを参考にされていますか？,GCI受講体験に関して、改善点・要望などがあればご自由にご記入ください。,ご感想などあればご自由にご記入ください。,GCI受講継続の妨げとなっている点・難しいと感じる点はありますか？（複数選択可）_Colab,GCI受講継続の妨げとなっている点・難しいと感じる点はありますか？（複数選択可）_Drive,GCI受講継続の妨げとなっている点・難しいと感じる点はありますか？（複数選択可）_Slack・Notion(手引き)・Drive内ドキュメントでの情報キャッチアップが困難,GCI受講継続の妨げとなっている点・難しいと感じる点はありますか？（複数選択可）_Slack等のツールの使用が困難,GCI受講継続の妨げとなっている点・難しいと感じる点はありますか？（複数選択可）_Zoom,GCI受講継続の妨げとなっている点・難しいと感じる点はありますか？（複数選択可）_その他,GCI受講継続の妨げとなっている点・難しいと感じる点はありますか？（複数選択可）_コンペが難しい,GCI受講継続の妨げとなっている点・難しいと感じる点はありますか？（複数選択可）_サポートや質問環境が不足している,GCI受講継続の妨げとなっている点・難しいと感じる点はありますか？（複数選択可）_モチベーションが続かない,GCI受講継続の妨げとなっている点・難しいと感じる点はありますか？（複数選択可）_周囲との進捗差に焦りを感じる,GCI受講継続の妨げとなっている点・難しいと感じる点はありますか？（複数選択可）_宿題が難しい,GCI受講継続の妨げとなっている点・難しいと感じる点はありますか？（複数選択可）_時間の確保が難しい,GCI受講継続の妨げとなっている点・難しいと感じる点はありますか？（複数選択可）_最終課題が難しい,GCI受講継続の妨げとなっている点・難しいと感じる点はありますか？（複数選択可）_期待していた内容とギャップがある,GCI受講継続の妨げとなっている点・難しいと感じる点はありますか？（複数選択可）_特にない,GCI受講継続の妨げとなっている点・難しいと感じる点はありますか？（複数選択可）_講義内容が難しい,LLM(ChatGPT等)をどんな用途で使用していますか？（複数選択可）_LLM(ChatGPT等)を使用していない,LLM(ChatGPT等)をどんな用途で使用していますか？（複数選択可）_その他,LLM(ChatGPT等)をどんな用途で使用していますか？（複数選択可）_コンペ・最終課題の解法の壁打ち,LLM(ChatGPT等)をどんな用途で使用していますか？（複数選択可）_コーディング補助,LLM(ChatGPT等)をどんな用途で使用していますか？（複数選択可）_講義内容の深堀り,LLM(ChatGPT等)をどんな用途で使用していますか？（複数選択可）_講義内容の疑問解決,Slackの活用について難しく感じる部分はありますか？（複数選択可）_1万人規模のSlackでの発言がはばかられる,Slackの活用について難しく感じる部分はありますか？（複数選択可）_Slackの使い方が難しい・慣れない,Slackの活用について難しく感じる部分はありますか？（複数選択可）_その他,Slackの活用について難しく感じる部分はありますか？（複数選択可）_情報量が多く、何を見たらいいかわからない,Slackの活用について難しく感じる部分はありますか？（複数選択可）_特に問題はない,Slackの活用について難しく感じる部分はありますか？（複数選択可）_疑問解決ができない/スムーズでない,Slackの活用について難しく感じる部分はありますか？（複数選択可）_重要な通知を見落としてしまう,Botからの通知はいかがでしょうか？(複数選択可)_Botからのメッセージが届いたことがない,Botからの通知はいかがでしょうか？(複数選択可)_その他,Botからの通知はいかがでしょうか？(複数選択可)_出欠アンケート以外のリマインドも送信してほしい,Botからの通知はいかがでしょうか？(複数選択可)_届いたことはあるが内容を確認していない,Botからの通知はいかがでしょうか？(複数選択可)_現在の通知内容で満足している,Botからの通知はいかがでしょうか？(複数選択可)_通知は煩わしいので不要である,Botからの通知はいかがでしょうか？(複数選択可)_通知内容が適切でない,講義冒頭の松尾先生が登場するGCI紹介動画の印象を教えて下さい。（複数選択可）_あまり必要だと感じない,講義冒頭の松尾先生が登場するGCI紹介動画の印象を教えて下さい。（複数選択可）_その他,講義冒頭の松尾先生が登場するGCI紹介動画の印象を教えて下さい。（複数選択可）_モチベーション維持につながっている,講義冒頭の松尾先生が登場するGCI紹介動画の印象を教えて下さい。（複数選択可）_今から授業が始まるという心の準備につながっている,講義冒頭の松尾先生が登場するGCI紹介動画の印象を教えて下さい。（複数選択可）_動画の間に受講環境を準備している,講義冒頭の松尾先生が登場するGCI紹介動画の印象を教えて下さい。（複数選択可）_東大松尾研の講義であることの認識につながっている,講義冒頭の松尾先生が登場するGCI紹介動画の印象を教えて下さい。（複数選択可）_特に印象には残っていない,GCI紹介動画の次に流れる、TAからのメッセージの印象を教えて下さい。(複数選択可)_TAを身近に感じられる,GCI紹介動画の次に流れる、TAからのメッセージの印象を教えて下さい。(複数選択可)_あまり必要だと感じない,GCI紹介動画の次に流れる、TAからのメッセージの印象を教えて下さい。(複数選択可)_その他,GCI紹介動画の次に流れる、TAからのメッセージの印象を教えて下さい。(複数選択可)_モチベーション維持につながっている,GCI紹介動画の次に流れる、TAからのメッセージの印象を教えて下さい。(複数選択可)_受講のヒントになっている,GCI紹介動画の次に流れる、TAからのメッセージの印象を教えて下さい。(複数選択可)_受講中のイメージがわく,GCI紹介動画の次に流れる、TAからのメッセージの印象を教えて下さい。(複数選択可)_受講後のイメージがわく,GCI紹介動画の次に流れる、TAからのメッセージの印象を教えて下さい。(複数選択可)_講義の全体像がよくわかる,講義資料に関して、わかりづらかった回があれば教えて下さい。（複数選択可）_わかりづらい講義資料はなかった,講義資料に関して、わかりづらかった回があれば教えて下さい。（複数選択可）_第1回 導入（データサイエンス・本講義の概要）,講義資料に関して、わかりづらかった回があれば教えて下さい。（複数選択可）_第2回 Python基礎（プログラミング文法／演算・条件分岐・関数）,講義資料に関して、わかりづらかった回があれば教えて下さい。（複数選択可）_第3回 Pythonによる科学計算（Numpy）,講義資料に関して、わかりづらかった回があれば教えて下さい。（複数選択可）_第4回 Pythonによるデータ加工処理の基礎（Pandas）,講義資料に関して、わかりづらかった回があれば教えて下さい。（複数選択可）_第5回 Pythonによるデータ可視化の基礎（Matplotlib）,講義資料に関して、わかりづら

In [44]:
try:
    # スプレッドシートを開く
    spreadsheet = gc.open_by_key(sheet_id)

    # --- 指定されたシート名が既に存在するかどうかを確認 ---
    try:
        # 同名のシートの取得を試みます
        existing_sheet = spreadsheet.worksheet(sheet_name_revised)

        # この行が実行された場合、シートは既に存在します
        print(f"❌ 書き込み停止: シート名 '{sheet_name_revised}' は既にスプレッドシート内に存在します。")
        print("重複しないように、コード内の変数 `sheet_name_revised` の値を変更してください。")

    except gspread.WorksheetNotFound:
        # 新しいシートを作成
        worksheet = spreadsheet.add_worksheet(title=sheet_name_revised, rows="1", cols="1")

        # データフレームを新しいシートに書き込みます
        set_with_dataframe(worksheet, df_new, include_index=False, resize=True)

        print(f"\n✅ 完了: データフレームをシート '{sheet_name_revised}' に正常に書き込みました。")
        print(f"スプレッドシートURL: https://docs.google.com/spreadsheets/d/{sheet_id}/")

except gspread.exceptions.SpreadsheetNotFound:
    print(f"エラー: スプレッドシートID '{sheet_id}' が見つかりません。")
    print("IDが正しいか、または認証したアカウントがそのスプレッドシートの編集権限を持っているか確認してください。")
except Exception as e:
    print(f"予期せぬエラーが発生しました: {e}")


✅ 完了: データフレームをシート '整形後' に正常に書き込みました。
スプレッドシートURL: https://docs.google.com/spreadsheets/d/1sCzqPptYkqMrT8Hx7tCvnS3TSNwI12PiupSaz_TkIOQ/
